In [ ]:
import pandas as pd

In [ ]:
import pickle

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [ ]:
df.head()

In [ ]:
rows = df.shape[0]
cols = df.shape[1]
rows


In [ ]:
cols

In [ ]:
print(df['tpep_pickup_datetime'].dt.time)

In [ ]:

df['duration']= df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"] 



In [ ]:
df['duration'] = df['duration'] / pd.Timedelta(minutes=1)

In [ ]:
df.head()

In [ ]:
df.describe()


In [ ]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [ ]:
new_rows = df.shape[0]
new_rows

In [ ]:
outliers = new_rows / rows
outliers

In [ ]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()


In [ ]:
train_dicts

In [ ]:

X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)



In [ ]:
def read_dataframe(filename):
 
    df = pd.read_parquet(filename)

    df['duration']= df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"] 
    df['duration'] = df['duration'] / pd.Timedelta(minutes=1)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [ ]:
len(df_train), len(df_val)

In [ ]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']


In [ ]:
df_train.head()

In [ ]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']


dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
y_val

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)